<a href="https://colab.research.google.com/github/jaffotsa/Guros/blob/main/Prueba_tecnica_Jaffet_Leon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Vamos a usar la forma automatizada de conectar la API de Spotify con Python, pero para ello usaré la librería de spotipy,
# la cual es sencilla de usar y simplifica la conexión con Spotify y el tratamiento directo de JSON's. Solo hago un pip
# para instalarle rápido.
!pip install spotipy

     |████████████████████████████████| 62 kB 454 kB/s 
     |████████████████████████████████| 138 kB 30.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
# Importo mi client_id y mi secret, datos proporcionados por la API WEB de Spotify.
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
cid = 'b23848d7084948d98a57555fa233b06d'
secret = 'ca9cd282dfe24635a8a739bd31a6471d'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager
=
client_credentials_manager)

In [ ]:
import spotipy

In [ ]:
# Ahora bien, solo resta importar la data a un pandas data frame de cual debemos respondes: ¿Cuáles son los 5 artistas más prometedores del Q3 2021?.
# Vamos a usar la playlist https://open.spotify.com/playlist/06Zas2YzEfhIldupJ7wdsc para obtener los top hits de septiembre (proxy a Q3 2021).

# La lógica de usar esta playlist en particular se debe a que el usuario grabielwell actualiza esta playlist semanalmente (no atiende a mis gustos ni a
# mis criterios musicales). No obstante, copie la playlist a mi cuenta para poder usarla debido a que hice logging con mi client_id de mi cuenta
# personal.

# Con este for solo traigo el track y su id, para despues incluirle mi user y el id de la playlist que copie de gabrielwell
def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids

ids = getTrackIDs('Jaffet León', '06Zas2YzEfhIldupJ7wdsc')

In [ ]:
# Ahora debemos obtener toda la info pertinente a album, artista con base en el track_id: hacemos el match por cada coincidencia
# y traerlo por variable individuales trayendolo con append a meta y features definidicas al comienzo.
def getTrackFeatures(id):
  meta = sp.track(id)
  features = sp.audio_features(id)

  # variables categóricas
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  release_date = meta['album']['release_date']
  length = meta['duration_ms']
  popularity = meta['popularity']

  # características
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']

  track = [name, album, artist, release_date, length, popularity, danceability, acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
  return track

In [ ]:
# Vamos a exportarlo a un CSV CON PANDAS y usamos de paso la library time para ampliar la escucha y que no colapse si superó el time nativo
# de la API:
import pandas as pd
import time 
tracks = []
for i in range(len(ids)):
  time.sleep(.10)
  track = getTrackFeatures(ids[i])
  tracks.append(track)

# crear dataset
df = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])
df.to_csv("spotify.csv", sep = ',')

In [ ]:
# Empecemos a ver como se comporta el DataFrame
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              100 non-null    object 
 1   album             100 non-null    object 
 2   artist            100 non-null    object 
 3   release_date      100 non-null    object 
 4   length            100 non-null    int64  
 5   popularity        100 non-null    int64  
 6   danceability      100 non-null    float64
 7   acousticness      100 non-null    float64
 8   danceability      100 non-null    float64
 9   energy            100 non-null    float64
 10  instrumentalness  100 non-null    float64
 11  liveness          100 non-null    float64
 12  loudness          100 non-null    float64
 13  speechiness       100 non-null    float64
 14  tempo             100 non-null    float64
 15  time_signature    100 non-null    int64  
dtypes: float64(9), int64(3), object(4)
memory usa

,length,popularity,danceability,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,205995.940000,65.970000,0.628680,0.261320,0.628680,0.628480,0.001016,0.170260,-6.077860,0.072867,120.856260,3.980000
std,40758.517286,16.711126,0.126037,0.272695,0.126037,0.167262,0.007458,0.117325,2.564051,0.068481,28.479347,0.245361
min,106880.000000,0.000000,0.310000,0.000191,0.310000,0.109000,0.000000,0.024600,-20.638000,0.025900,75.029000,3.000000
25%,176200.500000,59.000000,0.558250,0.033225,0.558250,0.511000,0.000000,0.096300,-7.288750,0.034175,97.753000,4.000000
50%,205696.500000,66.500000,0.640500,0.168000,0.640500,0.650500,0.000000,0.121500,-5.670500,0.045050,119.931000,4.000000
75%,231061.750000,77.250000,0.722500,0.435250,0.722500,0.776750,0.000016,0.211500,-4.475500,0.076075,134.554000,4.000000
max,352093.000000,97.000000,0.891000,0.948000,0.891000,0.913000,0.073900,0.694000,-1.512000,0.347000,201.025000,5.000000


In [ ]:
# Basado en ello saquemos los top artistas por popularity, graficando con matplotlib y pyplot
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import plotly 
import plotly.express as px

df_numbercharted=df.groupby('artist').sum().sort_values('popularity', ascending=False)
df_numbercharted=df_numbercharted.reset_index()
df_numbercharted

,artist,length,popularity,danceability,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,Taylor Swift,4065950,1046,8.271,4.411191,8.271,10.179,0.001815,2.7468,-101.441,0.6882,2086.732,64
1,Olivia Rodrigo,1070967,355,3.116,3.244000,3.116,3.017,0.000024,1.0580,-44.337,0.4755,810.932,24
2,Ariana Grande,869160,331,3.444,2.266000,3.444,2.785,0.000001,0.7091,-32.250,0.5405,605.070,20
3,Dua Lipa,814133,283,2.849,0.546930,2.849,3.099,0.000030,1.1140,-16.965,0.5372,449.909,16
4,Doja Cat,605599,240,2.254,0.555400,2.254,1.931,0.002488,0.3001,-16.464,0.1558,369.995,12
5,Billie Eilish,661961,193,1.788,2.379000,1.788,1.033,0.007869,0.4057,-30.962,0.3619,374.864,11
6,KAROL G,561253,160,1.189,0.316700,1.189,1.490,0.000000,0.4090,-8.683,0.2380,185.562,8
7,Bad Bunny,405440,155,1.446,0.351300,1.446,1.347,0.000000,0.7456,-9.842,0.3753,175.087,8
8,H.E.R.,450426,133,1.438,0.690000,1.438,0.909,0.000029,0.2840,-16.435,0.3974,232.856,9
9,The Chicks,467845,130,0.996,0.668000,0.996,0.992,0.000101,0.2833,-11.152,0.0577,229.865,8


In [ ]:
# Ya estan ordenados por popularidad. Aquellos más prometedores serían:
# Taylor Swift
# Olivia Rodrigo
# Ariana Grande

# Así ehmos llegado a la respuesta mediante una aproximación con una playlist que incluye a los top hits de Septiembre, mes el cual
# es parte del Q3 de 2021 requerido, gracias!

# Grafiquemos por popularidad
px.bar(x='artist', y='popularity', data_frame=df_numbercharted.head(7), title="Top 7 de artistas por popularidad")